In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from getpass import getpass
import pandas as pd
import datetime as dt
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title: Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)

In [ ]:
import os
today = [dt.date.today()]
log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Communities\UpdateCultureHistory"
log_file = os.path.join(log_path, "Update Culture History_" + str(today[0]) + str(".txt"))
out = open(log_file,'w')


for f in find_storymaps:
    print(f.title, f.id)
    try:
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            panel_geog = sections[2]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)","")
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            query = "CommunityName = '" + str(community.replace("'","''")) + "'"
            print(query)

            #Define the function to access fields within a layer --> provides a pandas dataframe of the data table
        def access_lyr(content,out_fld):
            base = content.layers
            lyr = base[0]
            topic_data = lyr.query(where="(CommunityName = '" + str(community) +"')" ,out_fields=out_fld)
            return topic_data.df
### GEOGRAPHY AND CLIMATE ###
        print("Updating climate and geography panel...")
        geog_base = gis.content.get("85b622ae19b04c7caa37c193b20fc4f3")
        geog_lookup = access_lyr(geog_base, 'Location, Climate, Latitude, Longitude, Sq_Mi_Land, Sq_Mi_Water')
        panel_geog["content"] = "<style type='text/css'>.dcra-blue {\n display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style><p><a class='dcra-blue' data-storymaps='MJ-ACTION-1537213609501' data-storymaps-type='navigate'>Back to Table of Contents</a></p><p>&nbsp;</p><p><b>Latitude, Longitude: </b>" + str(geog_lookup.Latitude[0]) + ', ' + str(geog_lookup.Longitude[0]) + '</p><p>&nbsp;</p><p>' + str(geog_lookup.Location[0]) + '</p><p>&nbsp;</p><p><b>Climate: </b>' + str(geog_lookup.Climate[0]) + '</p><p>&nbsp;</p><p><b>Square Miles of Land: </b>' + str(geog_lookup.Sq_Mi_Land[0]) + '</p><p><b>Square Miles of Water: </b>' + str(geog_lookup.Sq_Mi_Water[0]) + '</p>'

        csmap.save()
        
        print("Geography & climate information updated!!")
        
    except Exception as e:
        print("----ERROR WITH " + str(community) + " STORYMAP. EXCEPTION: " + str(e) + "----")
        out.write("----ERROR WITH " + str(community) + " STORYMAP. EXCEPTION: " + str(e) + "----")
        out.write("\n\n")
        continue
        
out.close()       
print("Storymap updates complete!")